In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
source_language = "en"
target_language = "yo" 
lc = False  
seed = 42  
tag = "baseline" 

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/MyDrive/Data Science Project/Neural-machine-translation/Tatoeba/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/MyDrive/Data Science Project/Neural-machine-translation/Tatoeba/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/MyDrive/Data Science Project/Neural-machine-translation/Tatoeba/en-yo-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 5.1 MB/s 


In [5]:
# # Downloading our corpus
# ! opus_read -d Tatoeba -s $src -t $tgt -wm moses -w tatoeba.$src tatoeba.$tgt -q

# # extract the corpus file
# ! gunzip Tatoeba_latest_xml_$src-$tgt.xml.gz

# Downloading our corpus
! opus_read -d GlobalVoices -s $src -t $tgt -wm moses -w globalvoices.$src globalvoices.$tgt -q

# extract the corpus file
! gunzip GlobalVoices_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/GlobalVoices/latest/xml/en-yo.xml.gz not found. The following files are available for downloading:

   0 KB https://object.pouta.csc.fi/OPUS-GlobalVoices/v2018q4/xml/en-yo.xml.gz
 357 MB https://object.pouta.csc.fi/OPUS-GlobalVoices/v2018q4/xml/en.zip
  25 KB https://object.pouta.csc.fi/OPUS-GlobalVoices/v2018q4/xml/yo.zip

 357 MB Total size
./GlobalVoices_latest_xml_en-yo.xml.gz ... 100% of 0 KB
./GlobalVoices_latest_xml_en.zip ... 100% of 357 MB
./GlobalVoices_latest_xml_yo.zip ... 100% of 25 KB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2022-07-26 13:49:46--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2022-07-26 13:49:46 (6.51 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2022-07-26 13:49:46--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-yo.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 2

In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [10]:
import pandas as pd

# TMX file to dataframe
source_file = 'globalvoices.' + source_language
target_file = 'globalvoices.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 0/132 lines since contained in test set.


,source_sentence,target_sentence
0,An all-female flight crew makes history in Moz...,Òṣìṣẹ ́ inú ọkọ ̀ òfuurufú olóbìrin nìkan wọ ì...
1,Mozambique 's first all-female crew | Photo us...,Ikọ ̀ awakọ ̀ òfuurufú olóbìrin àkọ ́ kọ ́ irú...
2,It is a historic day : that is how many Mozamb...,Ọjọ ́ ìtàn ni ọjọ ́ yìí : ojú yìí ni ogunlọ ́ ...


### Pre-processing and export

In [11]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [12]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=328f494212f9977f35fbc8681d12049b08cb43e1c59e7b9d749ba42dad683c0f
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [13]:
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Filtering function. Adjust pad to narrow down the candidate matches to within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [14]:
start_time = time.time()

with Pool(cpu_count()-1) as pool:
    scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
hours, rem = divmod(time.time() - start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# Filter out "almost overlapping samples"
df_pp = df_pp.assign(scores=scores)
df_pp = df_pp[df_pp['scores'] < 95]

done in 0.0h:0.0min:2.230165719985962seconds


In [16]:
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 20

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
The festivals can boost consumption , what ’ s wrong with that ?
Child : I will not celebrate Western festivals Christmas is not a Chinese people ’ s festival .
I am Chinese and I don ’ t celebrate Western festivals .
The commentary references the history of the Eight-Nation Alliance , a coalition formed in response to the Boxer Rebellion in China between 1899 and 1901 when Chinese peasants rose up against foreign , colonial , Christian rule and culture .
In their panel session , they credited Global Voices ' campaign for keeping them alive .
The company has forbidden the celebration of Western festivals .
Yet , the Zone9 bloggers redefined patriotism with both their words and actions .
A Weibo user expressed frustration :
If people of a nation are too enthusiastic in celebrating other nations ’ festivals , it indicates that the country is suffered from extremely serious cultural invasion .
In aviation there are few women , very few , this is not only here but in the w